In [8]:
import sys
import os

# Get the parent directory of the current folder

project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

In [9]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
from scipy.optimize import minimize
from tqdm import tqdm
from scipy.stats import t, multivariate_normal, multivariate_t
from scipy.special import logsumexp, gamma, gammaln
from numpy.linalg import inv, det, slogdet, LinAlgError
import plotly.express as px
from plotly.subplots import make_subplots
from numpy import tril_indices, triu_indices
from typing import Literal, Optional, Dict, Union

# from simulation import * 

from Functions.simulation_1 import * 
from Functions.gas_filter_5 import *
from Functions.kf_filter_5 import *

## $\Phi, Q, R$ Diagonal

In [10]:
# Explicit parameters with regressors
T = 1000
N = 3
K = 3

c_true_3x3 = np.array([0.2, -0.1, 0.4])

Phi_true_3x3_no_correl = np.array([[0.8, 0.0, 0.0],
                         [0.0, 0.9, 0.0],
                         [0.0, 0.0, 0.7]])

beta_true_3x3 = np.array([[1.0, -0.5, 0.2],
                          [-1.2, 0.8, 0.4],
                          [0.5, 1.5, -0.3]])

Q_true_3x3_no_correl = np.array([[0.7, 0.0, 0.0],
                                 [0.0, 0.5, 0.0],
                                 [0.0, 0.0, 0.6]])

R_true_3x3_no_correl = np.array([[0.4, 0.0, 0.0],
                                 [0.0, 0.8, 0.0],
                                 [0.0, 0.0, 0.5]])

# Run explicit simulation with regressors
y_sim_3x3_no_correl, mu_sim_3x3_no_correl, X_sim_3x3_no_correl = simulate_multivariate_state_space(
    T=T,
    N=N,
    c=c_true_3x3,
    Phi=Phi_true_3x3_no_correl,
    beta=beta_true_3x3,  # Explicitly include regressors
    Q=Q_true_3x3_no_correl,
    R=R_true_3x3_no_correl,
    # use_intercept=None,
    seed=8888,
    t_noise=False
)

In [11]:
gas_results_3x3_no_correl_1 = estimate_and_filter_gas(
    y=y_sim_3x3_no_correl,
    X=X_sim_3x3_no_correl, 
    phi_type="diagonal",
    kappa_type="full", 
    fix_nu=None)


mu_gas_3x3_no_correl_1 = gas_results_3x3_no_correl_1["mu_filtered"]
u_gas_3x3_no_correl_1 = gas_results_3x3_no_correl_1["u"]
resid_gas_no_correl_1 = gas_results_3x3_no_correl_1["resid"]
Kappa_est_3x3_no_correl_1 = gas_results_3x3_no_correl_1["Kappa"]

Estimated omega:
 [-1.36002479  0.91134565 -1.25388972]
Estimated Phi:
 [[0.82055031 0.         0.        ]
 [0.         0.93679411 0.        ]
 [0.         0.         0.67368211]]
Estimated beta:
 [[ 3.30918476 -2.75295366  2.78000089]
 [-1.21067675  0.81783468  0.36867997]
 [ 0.45690351  1.44528489 -0.2658343 ]]
Estimated Omega (from lambda):
 [[1.26864343 0.         0.        ]
 [0.         1.56732981 0.        ]
 [0.         0.         1.24373241]]
Estimated Kappa:
 [[ 0.675095    0.02540011 -0.00956334]
 [-0.03714708  0.70276713  0.01693203]
 [ 0.01243485  0.0698418   0.52357772]]
Estimated nu:
 217.82425267226245
Log-likelihood: -4723.334546560808
AIC: 9502.669093121616
BIC: 9670.847385015823


In [12]:
kf_results_3x3_no_correl  =  multivariate_KF_with_estimation(
    y=y_sim_3x3_no_correl,
    X=X_sim_3x3_no_correl,
    initial_params=None,
    verbose=True,
    # max_iter=1000
)

mu_kf_3x3_no_correl = kf_results_3x3_no_correl['mu_filtered']
P_kf_3x3_no_correl = kf_results_3x3_no_correl['P_filtered']
kalman_gain_kf_3x3_no_correl = kf_results_3x3_no_correl['kalman_gain']

✅ Kalman estimation completed successfully.
Estimated parameters:
omega (unconditional mean): [0.10830066 0.18673558 0.26131751]
Phi (persistence matrix): 
[[ 0.81817752  0.00614307  0.00783764]
 [-0.0092082   0.93751724  0.00328047]
 [ 0.01932636  0.03297722  0.64841784]]
beta: 
[[ 1.83449223 -1.95079031  1.26127245]
 [-1.20879941  0.81945124  0.37152799]
 [ 0.45644362  1.44580476 -0.26629747]]
Q (state noise covariance): 
[[0.61395658 0.00078355 0.00517243]
 [0.00078355 0.40177707 0.00944575]
 [0.00517243 0.00944575 0.66798409]]
R (observation noise covariance): 
[[0.46928997 0.00309982 0.03973499]
 [0.00309982 0.83324378 0.00117527]
 [0.03973499 0.00117527 0.4611948 ]]
Log-likelihood: -4730.379945935127
AIC: 9526.759891870253
BIC: 9724.970021602712


In [13]:
gas_specs = [
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_diag_diag_nufree"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_diag_diag_nu20"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_diag_diag_nu50"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_diag_diag_nu100"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_diag_diag_nu500"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": None, "label": "GAS_diag_full_nufree"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_full_nu5"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 10, "label": "GAS_diag_full_nu10"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 20, "label": "GAS_diag_full_nu20"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 50, "label": "GAS_diag_full_nu50"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 100, "label": "GAS_diag_full_nu100"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 500, "label": "GAS_diag_full_nu500"},
]


In [16]:
gas_specs = [
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": None, "label": "GAS_diag_diag_nufree"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_diag_nu5"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 10, "label": "GAS_diag_diag_nu10"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 20, "label": "GAS_diag_diag_nu20"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 50, "label": "GAS_diag_diag_nu50"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 100, "label": "GAS_diag_diag_nu100"},
    {"phi_type": "diagonal", "kappa_type": "diagonal", "fix_nu": 500, "label": "GAS_diag_diag_nu500"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": None, "label": "GAS_diag_full_nufree"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 5, "label": "GAS_diag_full_nu5"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 10, "label": "GAS_diag_full_nu10"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 20, "label": "GAS_diag_full_nu20"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 50, "label": "GAS_diag_full_nu50"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 100, "label": "GAS_diag_full_nu100"},
    {"phi_type": "diagonal", "kappa_type": "full", "fix_nu": 500, "label": "GAS_diag_full_nu500"},
]

In [17]:
from collections import defaultdict

spec_results = defaultdict(list)
target_success = 30
sim_counter = 0

while any(len(v) < target_success for v in spec_results.values()) or len(spec_results) == 0:
    sim_seed = 8888 + sim_counter
    sim_counter += 1
    # Simulate data
    y_sim, mu_sim, X_sim = simulate_multivariate_state_space(
        T=T, N=N, c=c_true_3x3, Phi=Phi_true_3x3_no_correl,
        beta=beta_true_3x3, Q=Q_true_3x3_no_correl, R=R_true_3x3_no_correl, seed=sim_seed
    )

    # KF
    try:
        if len(spec_results["KF"]) < target_success:
            kf_results = multivariate_KF_with_estimation(
            y=y_sim, 
            X=X_sim, 
            initial_params=None, 
            verbose=False,
            ftol=1e-8, 
            gtol=1e-6,  
        )
            # Calculate MSE for mu
            mu_kf = kf_results['mu_filtered']
            mse_kf = np.mean((mu_sim[1:] - mu_kf[1:]) ** 2)
            mae_kf = np.mean(np.abs(mu_sim[1:] - mu_kf[1:]))

            spec_results["KF"].append({
                "seed": sim_seed, "loglik": kf_results["loglik"],
                "aic": kf_results["aic"], "bic": kf_results["bic"],
                "mse": mse_kf, "mae": mae_kf
                })

    except Exception as e:
        print(f"KF failed: {e}")
    # GAS
    for spec in gas_specs:
        try:
            label = spec["label"]
            if len(spec_results[label]) < target_success:
                res_gas = estimate_and_filter_gas(
                    y=y_sim, 
                    X=X_sim,
                    phi_type=spec["phi_type"], 
                    kappa_type=spec["kappa_type"],
                    fix_nu=spec["fix_nu"], 
                    verbose=False,
                    ftol=1e-8, 
                    gtol=1e-6   
                )

                # Calculate MSE for mu
                mu_gas = res_gas["mu_filtered"]
                mse_gas = np.mean((mu_sim[1:] - mu_gas[1:]) ** 2)
                mae_gas = np.mean(np.abs(mu_sim[1:] - mu_gas[1:]))

                spec_results[label].append({
                    "seed": sim_seed, "loglik": res_gas["loglik"],
                    "aic": res_gas["aic"], "bic": res_gas["bic"],
                    "mse": mse_gas, "mae": mae_gas
                })
        except Exception as e:
            print(f"GAS {label} failed: {e}")

    # Break if all filled
    if all(len(v) >= target_success for v in spec_results.values()):
        break



In [ ]:
rows = []
for spec_label, run_list in spec_results.items():
    for run in run_list:
        row = {'spec': spec_label}
        row.update(run) 
        rows.append(row)

df_results = pd.DataFrame(rows)


agg = df_results.groupby('spec').agg({
    'loglik': ['mean', 'std'],
    'aic': ['mean', 'std'],
    'bic': ['mean', 'std'],
    'mse': ['mean', 'std'],  # Mean Squared Error
    'mae': ['mean', 'std'],  # Mean Absolute Error
}).reset_index()
agg.columns = ['spec', 'loglik_mean', 'loglik_std', 'aic_mean', 'aic_std', 'bic_mean', 'bic_std', 'mse_mean', 'mse_std', 'mae_mean', 'mae_std']  


agg

,spec,loglik_mean,loglik_std,aic_mean,aic_std,bic_mean,bic_std,mse_mean,mse_std,mae_mean,mae_std
0,GAS_diag_diag_nu10,-4746.751278,33.301314,9535.502556,66.602628,9661.636275,66.602628,1.147754,0.121201,0.902438,0.057777
1,GAS_diag_diag_nu100,-4710.390696,31.268732,9462.781391,62.537464,9588.915110,62.537464,0.708787,0.060632,0.682589,0.033338
2,GAS_diag_diag_nu20,-4721.812915,32.222175,9485.625830,64.444350,9611.759549,64.444350,0.924790,0.091952,0.795837,0.047022
3,GAS_diag_diag_nu5,-4805.363644,34.942071,9664.727287,69.884142,9826.899211,69.884142,1.418442,0.147707,1.017733,0.064970
4,GAS_diag_diag_nu50,-4712.075107,31.517919,9466.150213,63.035838,9592.283932,63.035838,0.762956,0.067969,0.711921,0.036690
5,GAS_diag_diag_nu500,-4709.860914,31.060511,9461.721829,62.121022,9587.855547,62.121022,0.663024,0.053590,0.657357,0.030037
6,GAS_diag_diag_nufree,-4709.893041,31.211083,9463.786082,62.422167,9595.926169,62.422167,14.720471,12.859350,3.411365,1.543058
7,GAS_diag_full_nu10,-4743.715435,32.989862,9541.430870,65.979723,9703.602795,65.979723,1.148620,0.121874,0.902617,0.057781
8,GAS_diag_full_nu100,-4707.738422,30.934779,9469.476844,61.869557,9631.648769,61.869557,0.710173,0.059529,0.683013,0.032693
9,GAS_diag_full_nu20,-4718.988894,31.887555,9491.977788,63.775109,9654.149713,63.775109,0.924974,0.091766,0.795578,0.046680


In [20]:
print(agg.to_latex())

\begin{tabular}{llrrrrrrrrrr}
\toprule
 & spec & loglik_mean & loglik_std & aic_mean & aic_std & bic_mean & bic_std & mse_mean & mse_std & mae_mean & mae_std \\
\midrule
0 & GAS_diag_diag_nu10 & -4746.751278 & 33.301314 & 9535.502556 & 66.602628 & 9661.636275 & 66.602628 & 1.147754 & 0.121201 & 0.902438 & 0.057777 \\
1 & GAS_diag_diag_nu100 & -4710.390696 & 31.268732 & 9462.781391 & 62.537464 & 9588.915110 & 62.537464 & 0.708787 & 0.060632 & 0.682589 & 0.033338 \\
2 & GAS_diag_diag_nu20 & -4721.812915 & 32.222175 & 9485.625830 & 64.444350 & 9611.759549 & 64.444350 & 0.924790 & 0.091952 & 0.795837 & 0.047022 \\
3 & GAS_diag_diag_nu5 & -4805.363644 & 34.942071 & 9664.727287 & 69.884142 & 9826.899211 & 69.884142 & 1.418442 & 0.147707 & 1.017733 & 0.064970 \\
4 & GAS_diag_diag_nu50 & -4712.075107 & 31.517919 & 9466.150213 & 63.035838 & 9592.283932 & 63.035838 & 0.762956 & 0.067969 & 0.711921 & 0.036690 \\
5 & GAS_diag_diag_nu500 & -4709.860914 & 31.060511 & 9461.721829 & 62.121022 & 9587.8